In [1]:
import numpy as np
from numpy import linalg as la
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
import cvxpy as cp
import random
import time
import os
from joblib import Parallel, delayed

from src.model import Nonneg_dagma, MetMulDagma
import src.utils as utils

from baselines.colide import colide_ev
from baselines.dagma_linear import DAGMA_linear
from baselines.notears import notears_linear

SEED = 10
N_CPUS = os.cpu_count()

# Set CPU limit for np
# os.environ["OPENBLAS_NUM_THREADS"] = str(N_CPUS)
# os.environ["MKL_NUM_THREADS"] = str(N_CPUS)
# os.environ["NUMBA_NUM_THREADS"] = str(N_CPUS)
# os.environ["OMP_NUM_THREADS"] = str(N_CPUS)
# os.environ["NUMEXP_NUM_THREADS"] = str(N_CPUS)
# os.environ["MKL_NUM_THREADS"] = str(N_CPUS)
# os.environ["OMP_NUM_THREADS"] = str(N_CPUS)

np.random.seed(SEED)
random.seed(SEED)

/home/srey/Investigacion/cvx_dag_learning/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_lamb_value(n_nodes, n_samples, times=1):
    return np.sqrt(np.log(n_nodes) / n_samples) * times 

def run_exps(g, data_p, exps, thr=.2, verb=False):
    A_true, _, X = utils.simulate_sem(**data_p)
    A_true_bin = utils.to_bin(A_true, thr)
    X_std = utils.standarize(X)

    M, N = X.shape

    fidelity = 1/data_p['n_samples']*la.norm(X - X @ A_true, 'fro')**2
    fidelity_norm = 1/data_p['n_samples']*la.norm(X_std - X_std @ A_true, 'fro')**2

    print(f'{g}: Fidelity: {fidelity:.3f}  -  Fidelity (norm): {fidelity_norm:.3f}')

    shd, fscore, err, acyc, runtime = [np.zeros(len(exps))  for _ in range(5)]
    for i, exp in enumerate(exps):
        X_aux = X_std if 'standarize' in exp.keys() and exp['standarize'] else X

        args = exp['args'].copy()
        if 'fix_lamb' in exp.keys() and not exp['fix_lamb']:
            args['lamb'] = get_lamb_value(N, M, args['lamb'])

        if 'know_var' in exp.keys() and exp['know_var']:
            args['Sigma'] = data_p['var']

        if exp['model'] == notears_linear:
            t_i = time.time()
            A_est = notears_linear(X_aux, **args)
            t_solved = time.time() - t_i
        else:
            model = exp['model'](**exp['init']) if 'init' in exp.keys() else exp['model']()
            t_i = time.time()
            model.fit(X_aux, **args)
            t_solved = time.time() - t_i

            A_est = model.W_est

        A_est_bin = utils.to_bin(A_est, thr)
        shd[i], _, _ = utils.count_accuracy(A_true_bin, A_est_bin)
        fscore[i] = f1_score(A_true_bin.flatten(), A_est_bin.flatten())
        err[i] = utils.compute_norm_sq_err(A_true, A_est)
        acyc[i] = model.dagness(A_est) if hasattr(model, 'dagness') else 1
        runtime[i] = t_solved

        if verb:
            print(f'\t-{g}: {exp["leg"]}: shd {shd[i]}  -  err: {err[i]:.3f}  -  time: {runtime[i]:.3f}')

    return shd, fscore, err, acyc, runtime

# Comparing models - Unit variance

In [4]:
Exps = [
    # Proj Grad Desc - Logdet
    ## 4 secs
    {'model': Nonneg_dagma, 'args': {'stepsize': 5e-3, 'alpha': .5, 's': 1, 'lamb': .1, 'max_iters': 20000, 'tol': 1e-5},
     'init': {'acyclicity': 'logdet', 'primal_opt': 'fista'}, 'fix_lamb': False, 'standarize': False, 'leg': 'PGD-FISTA'},
    ## 20 secs
    {'model': Nonneg_dagma, 'args': {'stepsize': 5e-3, 'alpha': .5, 's': 1, 'lamb': .1, 'max_iters': 20000, 'tol': 1e-5},
     'init': {'acyclicity': 'logdet', 'primal_opt': 'adam'}, 'fix_lamb': False, 'standarize': False, 'leg': 'PGD-Adam'},

    # Method of Multipliers - Logdet
    ## 3 mins
    # {'model': MetMulDagma, 'args': {'stepsize': 3e-4, 'alpha_0': .01, 'rho_0': .05, 's': 1, 'lamb': .1, 'iters_in': 40000,
    #  'iters_out': 10, 'beta': 2}, 'init': {'acyclicity': 'logdet', 'primal_opt': 'pgd'}, 'standarize': False, 
    #  'fix_lamb': False, 'leg': 'MM'},
    ## 40 secs
    {'model': MetMulDagma, 'args': {'stepsize': 5e-5, 'alpha_0': .5, 'rho_0': .5, 's': 1, 'lamb': .1, 'iters_in': 10000,
     'iters_out': 10, 'beta': 2}, 'init': {'acyclicity': 'logdet', 'primal_opt': 'fista'}, 'standarize': False,
     'fix_lamb': False, 'leg': 'MM-FISTA'},
    ## 35 secs
    {'model': MetMulDagma, 'args': {'stepsize': 3e-4, 'alpha_0': .01, 'rho_0': .05, 's': 1, 'lamb': .1, 'iters_in': 10000,
     'iters_out': 10, 'beta': 2}, 'init': {'acyclicity': 'logdet', 'primal_opt': 'adam'}, 'standarize': False,
     'fix_lamb': False, 'leg': 'MM-adam'},
    ## 35 secs
    {'model': MetMulDagma, 'args': {'stepsize': 3e-4, 'alpha_0': .01, 'rho_0': .05, 's': 1, 'lamb': .05, 'iters_in': 10000,
     'iters_out': 10, 'beta': 2}, 'init': {'acyclicity': 'logdet', 'primal_opt': 'adam'}, 'standarize': False,
     'fix_lamb': True, 'leg': 'MM-adam-fix'},

    # Proj Grad Desc - Mat exp
    ## 3 mins
    # {'model': Nonneg_dagma, 'args': {'stepsize': 5e-6, 'alpha': 1e5, 's': 1, 'lamb': 2e-4, 'max_iters': 20000, 'tol': 1e-5},
    #  'init': {'acyclicity': 'matexp', 'primal_opt': 'fista'}, 'fix_lamb': False, 'standarize': False, 'leg': 'PGD-matexp'},

    # Method of Multipliers - Mat exp
    ## 1.3 mins
    {'model': MetMulDagma, 'args': {'stepsize': 1e-5, 'alpha_0': 1, 'rho_0': 10, 's': 1, 'lamb': .1, 'iters_in': 10000,
     'iters_out': 10, 'beta': 10, 'tol': 1e-5}, 'init': {'acyclicity': 'matexp', 'primal_opt': 'fista'}, 'standarize': False,
     'fix_lamb': False, 'leg': 'MM-fista-matexp'},

    {'model': MetMulDagma, 'args': {'stepsize': 1e-4, 'alpha_0': 1, 'rho_0': 10, 's': 1, 'lamb': .05, 'iters_in': 10000,
     'iters_out': 5, 'beta': 10, 'tol': 1e-5}, 'init': {'acyclicity': 'matexp', 'primal_opt': 'adam'}, 'standarize': False,
     'fix_lamb': False, 'leg': 'MM-adam-matexp'},

    # Colide
    {'model': colide_ev, 'args': {'lambda1': .05, 'T': 4, 's': [1.0, .9, .8, .7], 'warm_iter': 2e4,
     'max_iter': 7e4, 'lr': .0003}, 'standarize': False, 'leg': 'CoLiDe'},

    # DAGMA
    {'model': DAGMA_linear, 'init': {'loss_type': 'l2'}, 'args': {'lambda1': .05, 'T': 4, 's': [1.0, .9, .8, .8],
     'warm_iter': 2e4, 'max_iter': 7e4, 'lr': .0003}, 'standarize': False, 'leg': 'DAGMA'},

    # NoTears
    {'model': notears_linear, 'args': {'loss_type': 'l2', 'lambda1': .1, 'max_iter': 10}, 'standarize': False, 'leg': 'NoTears'},
]

## 4N Edges

In [4]:
N = 100

n_dags = 20
verb = False
data_params = {
    'n_nodes': N,
    'n_samples': 500, # 1000,
    'graph_type': 'er',
    'edges': 4*N,
    'edge_type': 'positive',
    'w_range': (.5, 1),  # (.5, 1)
    'var': 1
}
print('CPUs employed:', N_CPUS)

t_init = time.time()
results = Parallel(n_jobs=N_CPUS)(delayed(run_exps)(g, data_params, Exps, thr=.2, verb=verb) for g in range(n_dags))
ellapsed_time = (time.time() - t_init)/60
print(f'----- Solved in {ellapsed_time:.3f} minutes -----')

# Extract results
shd, fscore, err, acyc, runtime = zip(*results)
metrics = {'shd': shd, 'fscore': fscore, 'err': err, 'acyc': acyc, 'time': runtime}


CPUs employed: 64


18: Fidelity: 100.870  -  Fidelity (norm): 384.893
10: Fidelity: 100.135  -  Fidelity (norm): 305.518
1: Fidelity: 99.928  -  Fidelity (norm): 377.734
2: Fidelity: 99.286  -  Fidelity (norm): 435.305
5: Fidelity: 99.055  -  Fidelity (norm): 320.810
14: Fidelity: 99.867  -  Fidelity (norm): 349.657
16: Fidelity: 99.006  -  Fidelity (norm): 397.524
3: Fidelity: 100.987  -  Fidelity (norm): 487.576
7: Fidelity: 99.992  -  Fidelity (norm): 357.989
9: Fidelity: 100.003  -  Fidelity (norm): 272.712
11: Fidelity: 100.215  -  Fidelity (norm): 241.727
15: Fidelity: 100.005  -  Fidelity (norm): 230.923
6: Fidelity: 100.075  -  Fidelity (norm): 448.086
8: Fidelity: 100.699  -  Fidelity (norm): 273.948
0: Fidelity: 99.588  -  Fidelity (norm): 383.460
13: Fidelity: 100.289  -  Fidelity (norm): 346.042
4: Fidelity: 100.492  -  Fidelity (norm): 336.830
19: Fidelity: 100.777  -  Fidelity (norm): 258.952
12: Fidelity: 99.982  -  Fidelity (norm): 341.102
17: Fidelity: 100.076  -  Fidelity (norm): 237.66

In [ ]:
exps_leg = [exp['leg'] for exp in Exps]
utils.display_results(exps_leg, metrics, agg='mean')
utils.display_results(exps_leg, metrics, agg='median')

,leg,shd,fscore,err,acyc,time
0,PGD-FISTA,23.4000 ± 69.9667,0.9669 ± 0.0985,0.0781 ± 0.2011,1.4382 ± 0.3440,5.2758 ± 1.1242
1,PGD-Adam,0.3000 ± 0.5568,0.9994 ± 0.0011,0.0125 ± 0.0012,1.5184 ± 0.2689,7.1093 ± 1.6227
2,MM-FISTA,9.9000 ± 25.2327,0.9852 ± 0.0369,0.0299 ± 0.0550,0.0020 ± 0.0015,13.1212 ± 6.8326
3,MM-adam,0.1500 ± 0.3571,0.9997 ± 0.0006,0.0102 ± 0.0009,0.0023 ± 0.0008,15.4722 ± 2.5948
4,MM-adam-fix,1.7500 ± 1.7854,0.9968 ± 0.0031,0.0183 ± 0.0048,0.0128 ± 0.0019,14.0607 ± 1.8537
5,MM-fista-matexp,29.7500 ± 11.0357,0.9564 ± 0.0152,0.0598 ± 0.0175,0.0000 ± 0.0000,21.4557 ± 3.2645
6,CoLiDe,27.6500 ± 12.2281,0.9564 ± 0.0172,0.0712 ± 0.0205,0.0003 ± 0.0000,20.0125 ± 2.6471
7,DAGMA,25.6500 ± 8.8390,0.9587 ± 0.0130,0.0719 ± 0.0166,0.0005 ± 0.0001,16.2532 ± 1.7175
8,NoTears,58.5500 ± 23.6251,0.9127 ± 0.0321,0.1508 ± 0.0564,0.0000 ± 0.0000,790.8892 ± 87.6780


,leg,shd,fscore,err,acyc,time
0,PGD-FISTA,0.0000 ± 69.9667,1.0000 ± 0.0985,0.0124 ± 0.2011,1.4877 ± 0.3440,5.2362 ± 1.1242
1,PGD-Adam,0.0000 ± 0.5568,1.0000 ± 0.0011,0.0124 ± 0.0012,1.5577 ± 0.2689,8.1323 ± 1.6227
2,MM-FISTA,0.5000 ± 25.2327,0.9994 ± 0.0369,0.0105 ± 0.0550,0.0021 ± 0.0015,11.3224 ± 6.8326
3,MM-adam,0.0000 ± 0.3571,1.0000 ± 0.0006,0.0100 ± 0.0009,0.0022 ± 0.0008,16.1359 ± 2.5948
4,MM-adam-fix,1.5000 ± 1.7854,0.9973 ± 0.0031,0.0171 ± 0.0048,0.0127 ± 0.0019,13.4242 ± 1.8537
5,MM-fista-matexp,31.0000 ± 11.0357,0.9558 ± 0.0152,0.0602 ± 0.0175,0.0000 ± 0.0000,21.6767 ± 3.2645
6,CoLiDe,25.5000 ± 12.2281,0.9573 ± 0.0172,0.0705 ± 0.0205,0.0003 ± 0.0000,19.8955 ± 2.6471
7,DAGMA,24.0000 ± 8.8390,0.9609 ± 0.0130,0.0715 ± 0.0166,0.0006 ± 0.0001,16.1166 ± 1.7175
8,NoTears,53.5000 ± 23.6251,0.9167 ± 0.0321,0.1324 ± 0.0564,0.0000 ± 0.0000,798.8911 ± 87.6780


# Changing variance

In [4]:
Exps = [
    {'model': MetMulDagma, 'args': {'stepsize': 3e-4, 'alpha_0': .01, 'rho_0': .05, 's': 1, 'lamb': .1, 'iters_in': 10000,
     'iters_out': 10, 'beta': 2}, 'init': {'acyclicity': 'logdet', 'primal_opt': 'adam'}, 'standarize': False,
     'fix_lamb': False, 'know_var': False, 'leg': 'MM-adam'},

    {'model': MetMulDagma, 'args': {'stepsize': 3e-4, 'alpha_0': .01, 'rho_0': .05, 's': 1, 'lamb': .1, 'iters_in': 10000,
     'iters_out': 10, 'beta': 2}, 'init': {'acyclicity': 'logdet', 'primal_opt': 'adam'}, 'standarize': False,
     'fix_lamb': False, 'know_var': True, 'leg': 'MM-adam-Sigma'},

    # Colide
    {'model': colide_ev, 'args': {'lambda1': .05, 'T': 4, 's': [1.0, .9, .8, .7], 'warm_iter': 2e4,
     'max_iter': 7e4, 'lr': .0003}, 'standarize': False, 'leg': 'CoLiDe'},

    # DAGMA
    {'model': DAGMA_linear, 'init': {'loss_type': 'l2'}, 'args': {'lambda1': .05, 'T': 4, 's': [1.0, .9, .8, .8],
     'warm_iter': 2e4, 'max_iter': 7e4, 'lr': .0003}, 'standarize': False, 'leg': 'DAGMA'},
]

## Var 0.5

In [12]:
N = 100

n_dags = 20
verb = False
data_params = {
    'n_nodes': N,
    'n_samples': 500, # 1000,
    'graph_type': 'er',
    'edges': 4*N,
    'edge_type': 'positive',
    'w_range': (.5, 1.5),  # (.5, 1)
    'var': .5
}
print('CPUs employed:', N_CPUS)

t_init = time.time()
results = Parallel(n_jobs=N_CPUS)(delayed(run_exps)(g, data_params, Exps, thr=.2, verb=verb) for g in range(n_dags))
ellapsed_time = (time.time() - t_init)/60
print(f'----- Solved in {ellapsed_time:.3f} minutes -----')

# Extract results
shd, fscore, err, acyc, runtime = zip(*results)
metrics = {'shd': shd, 'fscore': fscore, 'err': err, 'acyc': acyc, 'time': runtime}


CPUs employed: 64
0: Fidelity: 49.578  -  Fidelity (norm): 776.780
12: Fidelity: 50.081  -  Fidelity (norm): 889.655
14: Fidelity: 50.259  -  Fidelity (norm): 730.314
8: Fidelity: 50.429  -  Fidelity (norm): 980.756
13: Fidelity: 49.888  -  Fidelity (norm): 1142.881
11: Fidelity: 50.102  -  Fidelity (norm): 1020.700
19: Fidelity: 49.166  -  Fidelity (norm): 720.677
9: Fidelity: 50.246  -  Fidelity (norm): 592.648
18: Fidelity: 49.776  -  Fidelity (norm): 1031.619
17: Fidelity: 50.177  -  Fidelity (norm): 797.698
7: Fidelity: 50.047  -  Fidelity (norm): 725.051
2: Fidelity: 50.071  -  Fidelity (norm): 665.272
16: Fidelity: 50.311  -  Fidelity (norm): 1060.692
3: Fidelity: 50.246  -  Fidelity (norm): 888.261
15: Fidelity: 49.741  -  Fidelity (norm): 630.687
5: Fidelity: 50.230  -  Fidelity (norm): 884.208
10: Fidelity: 49.985  -  Fidelity (norm): 844.608
6: Fidelity: 49.857  -  Fidelity (norm): 1104.237
1: Fidelity: 49.757  -  Fidelity (norm): 786.457
4: Fidelity: 50.112  -  Fidelity (no

In [13]:
exps_leg = [exp['leg'] for exp in Exps]
utils.display_results(exps_leg, metrics, agg='mean')
utils.display_results(exps_leg, metrics, agg='median')

,leg,shd,fscore,err,acyc,time
0,MM-adam,5.8000 ± 5.9632,0.9915 ± 0.0078,0.0117 ± 0.0059,0.0013 ± 0.0012,26.1743 ± 11.0914
1,MM-adam-Sigma,5.7000 ± 8.9168,0.9931 ± 0.0104,0.0094 ± 0.0050,0.0015 ± 0.0025,27.9858 ± 11.9789
2,CoLiDe,53.0000 ± 39.3179,0.9280 ± 0.0469,0.0984 ± 0.0559,0.0002 ± 0.0001,45.4187 ± 11.6805
3,DAGMA,64.0500 ± 41.0676,0.9131 ± 0.0496,0.1301 ± 0.0654,0.0003 ± 0.0001,32.6522 ± 8.7436


,leg,shd,fscore,err,acyc,time
0,MM-adam,5.0000 ± 5.9632,0.9920 ± 0.0078,0.0107 ± 0.0059,0.0010 ± 0.0012,20.8458 ± 11.0914
1,MM-adam-Sigma,0.5000 ± 8.9168,0.9994 ± 0.0104,0.0065 ± 0.0050,0.0010 ± 0.0025,25.7074 ± 11.9789
2,CoLiDe,45.5000 ± 39.3179,0.9342 ± 0.0469,0.0942 ± 0.0559,0.0002 ± 0.0001,43.0157 ± 11.6805
3,DAGMA,51.5000 ± 41.0676,0.9251 ± 0.0496,0.1119 ± 0.0654,0.0003 ± 0.0001,31.1184 ± 8.7436


## Var 1

In [14]:
N = 100

n_dags = 20
verb = False
data_params = {
    'n_nodes': N,
    'n_samples': 500, # 1000,
    'graph_type': 'er',
    'edges': 4*N,
    'edge_type': 'positive',
    'w_range': (.5, 1.5),  # (.5, 1)
    'var': 1
}
print('CPUs employed:', N_CPUS)

t_init = time.time()
results = Parallel(n_jobs=N_CPUS)(delayed(run_exps)(g, data_params, Exps, thr=.2, verb=verb) for g in range(n_dags))
ellapsed_time = (time.time() - t_init)/60
print(f'----- Solved in {ellapsed_time:.3f} minutes -----')

# Extract results
shd, fscore, err, acyc, runtime = zip(*results)
metrics = {'shd': shd, 'fscore': fscore, 'err': err, 'acyc': acyc, 'time': runtime}


CPUs employed: 64
0: Fidelity: 100.616  -  Fidelity (norm): 739.946
17: Fidelity: 100.004  -  Fidelity (norm): 637.339
16: Fidelity: 99.829  -  Fidelity (norm): 578.626
19: Fidelity: 100.233  -  Fidelity (norm): 964.598
7: Fidelity: 100.004  -  Fidelity (norm): 1019.123
13: Fidelity: 100.879  -  Fidelity (norm): 869.161
11: Fidelity: 99.981  -  Fidelity (norm): 654.594
5: Fidelity: 99.601  -  Fidelity (norm): 820.977
10: Fidelity: 100.498  -  Fidelity (norm): 701.851
6: Fidelity: 100.033  -  Fidelity (norm): 817.921
1: Fidelity: 101.212  -  Fidelity (norm): 793.830
15: Fidelity: 100.176  -  Fidelity (norm): 863.089
14: Fidelity: 100.215  -  Fidelity (norm): 669.501
3: Fidelity: 99.980  -  Fidelity (norm): 945.485
12: Fidelity: 99.062  -  Fidelity (norm): 983.501
2: Fidelity: 100.058  -  Fidelity (norm): 954.360
18: Fidelity: 100.579  -  Fidelity (norm): 707.196
9: Fidelity: 99.428  -  Fidelity (norm): 987.294
8: Fidelity: 101.173  -  Fidelity (norm): 919.128
4: Fidelity: 100.278  -  Fi

In [15]:
exps_leg = [exp['leg'] for exp in Exps]
utils.display_results(exps_leg, metrics, agg='mean')
utils.display_results(exps_leg, metrics, agg='median')

,leg,shd,fscore,err,acyc,time
0,MM-adam,1.7000 ± 2.9513,0.9979 ± 0.0036,0.0068 ± 0.0018,0.0009 ± 0.0006,24.2674 ± 7.7007
1,MM-adam-Sigma,1.7000 ± 2.9513,0.9979 ± 0.0036,0.0068 ± 0.0018,0.0009 ± 0.0006,24.4109 ± 7.7620
2,CoLiDe,51.1000 ± 27.7793,0.9309 ± 0.0336,0.0890 ± 0.0361,0.0003 ± 0.0001,44.6155 ± 11.5792
3,DAGMA,53.9000 ± 24.9698,0.9270 ± 0.0314,0.0957 ± 0.0358,0.0007 ± 0.0001,33.6309 ± 7.9396


,leg,shd,fscore,err,acyc,time
0,MM-adam,0.0000 ± 2.9513,1.0000 ± 0.0036,0.0063 ± 0.0018,0.0009 ± 0.0006,20.9053 ± 7.7007
1,MM-adam-Sigma,0.0000 ± 2.9513,1.0000 ± 0.0036,0.0063 ± 0.0018,0.0009 ± 0.0006,21.0122 ± 7.7620
2,CoLiDe,56.0000 ± 27.7793,0.9218 ± 0.0336,0.0934 ± 0.0361,0.0003 ± 0.0001,42.2343 ± 11.5792
3,DAGMA,50.0000 ± 24.9698,0.9324 ± 0.0314,0.0977 ± 0.0358,0.0007 ± 0.0001,32.3228 ± 7.9396


## Var 2

In [16]:
N = 100

n_dags = 20
verb = False
data_params = {
    'n_nodes': N,
    'n_samples': 500, # 1000,
    'graph_type': 'er',
    'edges': 4*N,
    'edge_type': 'positive',
    'w_range': (.5, 1.5),  # (.5, 1)
    'var': 2
}
print('CPUs employed:', N_CPUS)

t_init = time.time()
results = Parallel(n_jobs=N_CPUS)(delayed(run_exps)(g, data_params, Exps, thr=.2, verb=verb) for g in range(n_dags))
ellapsed_time = (time.time() - t_init)/60
print(f'----- Solved in {ellapsed_time:.3f} minutes -----')

# Extract results
shd, fscore, err, acyc, runtime = zip(*results)
metrics = {'shd': shd, 'fscore': fscore, 'err': err, 'acyc': acyc, 'time': runtime}


CPUs employed: 64
0: Fidelity: 198.333  -  Fidelity (norm): 786.269
1: Fidelity: 198.333  -  Fidelity (norm): 861.800
2: Fidelity: 198.333  -  Fidelity (norm): 619.829
4: Fidelity: 198.333  -  Fidelity (norm): 740.772
3: Fidelity: 198.333  -  Fidelity (norm): 717.674
5: Fidelity: 198.333  -  Fidelity (norm): 741.107
6: Fidelity: 198.333  -  Fidelity (norm): 881.190
7: Fidelity: 198.333  -  Fidelity (norm): 711.794
8: Fidelity: 198.333  -  Fidelity (norm): 679.562
12: Fidelity: 198.333  -  Fidelity (norm): 720.798
9: Fidelity: 198.333  -  Fidelity (norm): 888.203
11: Fidelity: 198.333  -  Fidelity (norm): 862.091
10: Fidelity: 198.333  -  Fidelity (norm): 758.462
13: Fidelity: 198.333  -  Fidelity (norm): 682.477
15: Fidelity: 198.333  -  Fidelity (norm): 923.504
14: Fidelity: 198.333  -  Fidelity (norm): 623.578
16: Fidelity: 198.333  -  Fidelity (norm): 579.326
17: Fidelity: 198.333  -  Fidelity (norm): 1221.722
18: Fidelity: 198.333  -  Fidelity (norm): 646.192
19: Fidelity: 198.333 

In [17]:
exps_leg = [exp['leg'] for exp in Exps]
utils.display_results(exps_leg, metrics, agg='mean')
utils.display_results(exps_leg, metrics, agg='median')

,leg,shd,fscore,err,acyc,time
0,MM-adam,4.5500 ± 4.3986,0.9937 ± 0.0058,0.0116 ± 0.0040,0.0008 ± 0.0005,26.3615 ± 10.0382
1,MM-adam-Sigma,2.6000 ± 3.9294,0.9966 ± 0.0052,0.0082 ± 0.0039,0.0008 ± 0.0005,23.1975 ± 7.2022
2,CoLiDe,58.2500 ± 25.0057,0.9238 ± 0.0277,0.0872 ± 0.0243,0.0006 ± 0.0001,46.1573 ± 10.1057
3,DAGMA,76.6000 ± 24.7253,0.9031 ± 0.0266,0.1087 ± 0.0244,0.0018 ± 0.0002,35.2677 ± 6.6239


,leg,shd,fscore,err,acyc,time
0,MM-adam,2.5000 ± 4.3986,0.9950 ± 0.0058,0.0098 ± 0.0040,0.0007 ± 0.0005,23.2151 ± 10.0382
1,MM-adam-Sigma,0.0000 ± 3.9294,1.0000 ± 0.0052,0.0067 ± 0.0039,0.0007 ± 0.0005,20.8582 ± 7.2022
2,CoLiDe,52.0000 ± 25.0057,0.9264 ± 0.0277,0.0891 ± 0.0243,0.0006 ± 0.0001,43.4418 ± 10.1057
3,DAGMA,66.0000 ± 24.7253,0.9129 ± 0.0266,0.1009 ± 0.0244,0.0017 ± 0.0002,33.2738 ± 6.6239


## Var 5

In [18]:
N = 100

n_dags = 20
verb = False
data_params = {
    'n_nodes': N,
    'n_samples': 500, # 1000,
    'graph_type': 'er',
    'edges': 4*N,
    'edge_type': 'positive',
    'w_range': (.5, 1.5),  # (.5, 1)
    'var': 5
}
print('CPUs employed:', N_CPUS)

t_init = time.time()
results = Parallel(n_jobs=N_CPUS)(delayed(run_exps)(g, data_params, Exps, thr=.2, verb=verb) for g in range(n_dags))
ellapsed_time = (time.time() - t_init)/60
print(f'----- Solved in {ellapsed_time:.3f} minutes -----')

# Extract results
shd, fscore, err, acyc, runtime = zip(*results)
metrics = {'shd': shd, 'fscore': fscore, 'err': err, 'acyc': acyc, 'time': runtime}


CPUs employed: 64
0: Fidelity: 495.832  -  Fidelity (norm): 726.530
1: Fidelity: 495.832  -  Fidelity (norm): 796.900
2: Fidelity: 495.832  -  Fidelity (norm): 787.623
3: Fidelity: 495.832  -  Fidelity (norm): 790.322
4: Fidelity: 495.832  -  Fidelity (norm): 857.198
5: Fidelity: 495.832  -  Fidelity (norm): 721.912
6: Fidelity: 495.832  -  Fidelity (norm): 850.332
7: Fidelity: 495.832  -  Fidelity (norm): 1153.514
8: Fidelity: 495.832  -  Fidelity (norm): 680.370
10: Fidelity: 495.832  -  Fidelity (norm): 686.289
9: Fidelity: 495.832  -  Fidelity (norm): 735.539
12: Fidelity: 495.832  -  Fidelity (norm): 553.936
11: Fidelity: 495.832  -  Fidelity (norm): 841.051
13: Fidelity: 495.832  -  Fidelity (norm): 745.294
14: Fidelity: 495.832  -  Fidelity (norm): 611.547
16: Fidelity: 499.571  -  Fidelity (norm): 866.006
17: Fidelity: 499.497  -  Fidelity (norm): 740.742
15: Fidelity: 495.329  -  Fidelity (norm): 677.923
19: Fidelity: 501.765  -  Fidelity (norm): 497.747
18: Fidelity: 497.178 

In [19]:
exps_leg = [exp['leg'] for exp in Exps]
utils.display_results(exps_leg, metrics, agg='mean')
utils.display_results(exps_leg, metrics, agg='median')

,leg,shd,fscore,err,acyc,time
0,MM-adam,13.4000 ± 7.4256,0.9827 ± 0.0084,0.0174 ± 0.0031,0.0111 ± 0.0303,28.2781 ± 6.4358
1,MM-adam-Sigma,3.6500 ± 6.1423,0.9952 ± 0.0081,0.0088 ± 0.0063,0.0021 ± 0.0043,22.2940 ± 5.1122
2,CoLiDe,65.5500 ± 22.7145,0.9171 ± 0.0250,0.0873 ± 0.0202,0.0011 ± 0.0001,44.3546 ± 10.8711
3,DAGMA,126.4000 ± 30.5915,0.8559 ± 0.0300,0.1459 ± 0.0237,0.0071 ± 0.0005,34.3446 ± 6.5876


,leg,shd,fscore,err,acyc,time
0,MM-adam,12.0000 ± 7.4256,0.9835 ± 0.0084,0.0170 ± 0.0031,0.0024 ± 0.0303,26.4105 ± 6.4358
1,MM-adam-Sigma,0.5000 ± 6.1423,0.9994 ± 0.0081,0.0063 ± 0.0063,0.0011 ± 0.0043,20.9710 ± 5.1122
2,CoLiDe,60.0000 ± 22.7145,0.9179 ± 0.0250,0.0883 ± 0.0202,0.0011 ± 0.0001,40.1500 ± 10.8711
3,DAGMA,129.5000 ± 30.5915,0.8510 ± 0.0300,0.1458 ± 0.0237,0.0071 ± 0.0005,33.0292 ± 6.5876


## 2N Edges - 100 nodes

In [9]:
###  1000 samples - 100 nodes  #####
N = 100

n_dags = 25
data_params = {
    'n_nodes': N,
    'n_samples': 500, # 1000,
    'graph_type': 'er',
    'edges': 2*N,
    'edge_type': 'positive',
    'w_range': (.5, 1),
    'var': 1,
}

print('CPUs employed:', N_CPUS)

t_init = time.time()
results = Parallel(n_jobs=N_CPUS)(delayed(run_exps)(g, data_params, Exps, thr=.2, verb=False) for g in range(n_dags))
ellapsed_time = (time.time() - t_init)/60
print(f'----- Solved in {ellapsed_time:.3f} minutes -----')

# Extract results
shd, fscore, err, acyc, runtime = zip(*results)
metrics = {'shd': shd, 'fscore': fscore, 'err': err, 'acyc': acyc, 'time': runtime}


CPUs employed: 64
24: Fidelity: 99.942  -  Fidelity (norm): 82.769
13: Fidelity: 99.869  -  Fidelity (norm): 81.932
2: Fidelity: 100.775  -  Fidelity (norm): 75.372
6: Fidelity: 99.932  -  Fidelity (norm): 86.597
16: Fidelity: 100.415  -  Fidelity (norm): 89.817
22: Fidelity: 100.152  -  Fidelity (norm): 100.781
19: Fidelity: 100.168  -  Fidelity (norm): 73.771
9: Fidelity: 99.947  -  Fidelity (norm): 85.174
23: Fidelity: 99.184  -  Fidelity (norm): 77.518
8: Fidelity: 100.250  -  Fidelity (norm): 87.759
11: Fidelity: 100.081  -  Fidelity (norm): 79.250
18: Fidelity: 99.278  -  Fidelity (norm): 74.659
21: Fidelity: 100.237  -  Fidelity (norm): 101.3153: Fidelity: 100.314  -  Fidelity (norm): 83.436

20: Fidelity: 101.157  -  Fidelity (norm): 84.975
12: Fidelity: 99.168  -  Fidelity (norm): 86.761
7: Fidelity: 100.724  -  Fidelity (norm): 81.553
10: Fidelity: 99.222  -  Fidelity (norm): 81.092
0: Fidelity: 99.163  -  Fidelity (norm): 82.772
14: Fidelity: 98.947  -  Fidelity (norm): 91.7

In [ ]:
# M = 1000
exps_leg = [exp['leg'] for exp in Exps]
utils.display_results(exps_leg, metrics, agg='mean')
utils.display_results(exps_leg, metrics, agg='median')

,leg,shd,fscore,err,acyc,time
0,PGD-FISTA,5.9200 ± 20.4096,0.9861 ± 0.0479,0.0396 ± 0.0845,1.1838 ± 1.8126,1.7413 ± 1.7735
1,PGD-Adam,0.4000 ± 0.8000,0.9984 ± 0.0036,0.0241 ± 0.0050,0.3336 ± 0.2484,1.3293 ± 0.5644
2,MM-FISTA,0.4800 ± 1.4730,0.9984 ± 0.0042,0.0171 ± 0.0039,0.0034 ± 0.0010,10.9160 ± 1.3647
3,MM-adam,0.3600 ± 0.7940,0.9984 ± 0.0034,0.0172 ± 0.0033,0.0053 ± 0.0008,9.6703 ± 1.5310
4,MM-adam-fix,1.0400 ± 1.2800,0.9953 ± 0.0060,0.0148 ± 0.0072,0.0083 ± 0.0015,9.7200 ± 1.6303
5,MM-fista-matexp,23.2400 ± 9.7623,0.9301 ± 0.0303,0.0907 ± 0.0343,0.0000 ± 0.0000,18.0444 ± 1.8001
6,CoLiDe,6.3200 ± 5.3120,0.9771 ± 0.0189,0.0414 ± 0.0287,0.0001 ± 0.0000,13.4504 ± 1.2093
7,DAGMA,5.4000 ± 5.3442,0.9800 ± 0.0190,0.0393 ± 0.0280,0.0001 ± 0.0000,12.6154 ± 1.5806
8,NoTears,7.0400 ± 5.7514,0.9737 ± 0.0206,0.0445 ± 0.0331,0.0000 ± 0.0000,205.6684 ± 27.1424


,leg,shd,fscore,err,acyc,time
0,PGD-FISTA,0.0000 ± 20.4096,1.0000 ± 0.0479,0.0168 ± 0.0845,0.8227 ± 1.8126,1.0837 ± 1.7735
1,PGD-Adam,0.0000 ± 0.8000,1.0000 ± 0.0036,0.0233 ± 0.0050,0.2699 ± 0.2484,1.1390 ± 0.5644
2,MM-FISTA,0.0000 ± 1.4730,1.0000 ± 0.0042,0.0160 ± 0.0039,0.0035 ± 0.0010,10.8465 ± 1.3647
3,MM-adam,0.0000 ± 0.7940,1.0000 ± 0.0034,0.0163 ± 0.0033,0.0053 ± 0.0008,9.1744 ± 1.5310
4,MM-adam-fix,1.0000 ± 1.2800,0.9957 ± 0.0060,0.0129 ± 0.0072,0.0084 ± 0.0015,9.0627 ± 1.6303
5,MM-fista-matexp,22.0000 ± 9.7623,0.9267 ± 0.0303,0.0885 ± 0.0343,0.0000 ± 0.0000,17.6673 ± 1.8001
6,CoLiDe,6.0000 ± 5.3120,0.9780 ± 0.0189,0.0349 ± 0.0287,0.0001 ± 0.0000,13.2690 ± 1.2093
7,DAGMA,4.0000 ± 5.3442,0.9808 ± 0.0190,0.0353 ± 0.0280,0.0001 ± 0.0000,12.2461 ± 1.5806
8,NoTears,5.0000 ± 5.7514,0.9783 ± 0.0206,0.0347 ± 0.0331,0.0000 ± 0.0000,206.9690 ± 27.1424


# Comparing models - Standarized

In [9]:
Exps = [
    # Proj Grad Desc
    {'model': Nonneg_dagma, 'args': {'stepsize': 5e-3, 'alpha': .5, 's': 1, 'lamb': .1, 'max_iters': 20000, 'tol': 1e-5},
     'init': {'acyclicity': 'logdet', 'primal_opt': 'fista'}, 'fix_lamb': False, 'standarize': True, 'leg': 'PGD-FISTA'},

    {'model': Nonneg_dagma, 'args': {'stepsize': 5e-3, 'alpha': .5, 's': 1, 'lamb': .1, 'max_iters': 20000, 'tol': 1e-5},
     'init': {'acyclicity': 'logdet', 'primal_opt': 'adam'}, 'fix_lamb': False, 'standarize': True, 'leg': 'PGD-Adam'},

    # Method of Multipliers
    {'model': MetMulDagma, 'args': {'stepsize': 3e-4, 'alpha_0': .01, 'rho_0': .05, 's': 1, 'lamb': .1, 'iters_in': 40000,
     'iters_out': 10, 'beta': 2}, 'init': {'acyclicity': 'logdet', 'primal_opt': 'pgd'}, 'standarize': True, 
     'fix_lamb': False, 'leg': 'MM'},

    {'model': MetMulDagma, 'args': {'stepsize': 5e-5, 'alpha_0': .5, 'rho_0': .5, 's': 1, 'lamb': .1, 'iters_in': 10000,
     'iters_out': 10, 'beta': 2}, 'init': {'acyclicity': 'logdet', 'primal_opt': 'fista'}, 'standarize': True,
     'fix_lamb': False, 'leg': 'MM-FISTA'},

    {'model': MetMulDagma, 'args': {'stepsize': 3e-4, 'alpha_0': .01, 'rho_0': .05, 's': 1, 'lamb': .1, 'iters_in': 10000,
     'iters_out': 10, 'beta': 2}, 'init': {'acyclicity': 'logdet', 'primal_opt': 'adam'}, 'standarize': True,
     'fix_lamb': False, 'leg': 'MM-adam'},

    {'model': MetMulDagma, 'args': {'stepsize': 3e-4, 'alpha_0': .01, 'rho_0': .05, 's': 1, 'lamb': .05, 'iters_in': 10000,
     'iters_out': 10, 'beta': 2}, 'init': {'acyclicity': 'logdet', 'primal_opt': 'adam'}, 'standarize': True,
     'fix_lamb': True, 'leg': 'MM-adam-fix'},

    # Proj Grad Desc - Mat exp
    {'model': Nonneg_dagma, 'args': {'stepsize': 5e-6, 'alpha': 1e5, 's': 1, 'lamb': 2e-4, 'max_iters': 20000, 'tol': 1e-5},
     'init': {'acyclicity': 'matexp', 'primal_opt': 'fista'}, 'fix_lamb': False, 'standarize': True, 'leg': 'PGD-matexp'},

    # Method of Multipliers - Mat exp
    {'model': MetMulDagma, 'args': {'stepsize': 1e-4, 'alpha_0': 1, 'rho_0': 10, 's': 1, 'lamb': .05, 'iters_in': 10000,
     'iters_out': 10, 'beta': 10, 'tol': 1e-5}, 'init': {'acyclicity': 'matexp', 'primal_opt': 'fista'}, 'standarize': True,
     'fix_lamb': True, 'leg': 'MM-fista-matexp'},

    # Colide
    {'model': colide_ev, 'args': {'lambda1': .05, 'T': 4, 's': [1.0, .9, .8, .7], 'warm_iter': 2e4,
     'max_iter': 7e4, 'lr': .0003}, 'standarize': True, 'leg': 'CoLiDe'},

    # DAGMA
    {'model': DAGMA_linear, 'init': {'loss_type': 'l2'}, 'args': {'lambda1': .05, 'T': 4, 's': [1.0, .9, .8, .8],
     'warm_iter': 2e4, 'max_iter': 7e4, 'lr': .0003}, 'standarize': True, 'leg': 'DAGMA'},

    # NoTears
    {'model': notears_linear, 'args': {'loss_type': 'l2', 'lambda1': .1, 'max_iter': 10}, 'standarize': True, 'leg': 'NoTears'},
]

## 4N Edges

In [10]:
N = 100

n_dags = 20
verb=False
data_params = {
    'n_nodes': N,
    'n_samples': 500, # 1000,
    'graph_type': 'er',
    'edges': 4*N,
    'edge_type': 'positive',
    'w_range': (.5, 1),
    'var': 1
}

print('CPUs employed:', N_CPUS)

t_init = time.time()
results = Parallel(n_jobs=N_CPUS)(delayed(run_exps)(g, data_params, Exps, thr=.2, verb=verb) for g in range(n_dags))
ellapsed_time = (time.time() - t_init)/60
print(f'----- Solved in {ellapsed_time:.3f} seminutesconds -----')

# Extract results
shd, fscore, err, acyc, runtime = zip(*results)
metrics = {'shd': shd, 'fscore': fscore, 'err': err, 'acyc': acyc, 'time': runtime}


CPUs employed: 64
0: Fidelity: 99.166  -  Fidelity (norm): 310.793
2: Fidelity: 99.166  -  Fidelity (norm): 271.933
3: Fidelity: 99.166  -  Fidelity (norm): 401.871
1: Fidelity: 99.166  -  Fidelity (norm): 329.789
5: Fidelity: 99.166  -  Fidelity (norm): 331.198
4: Fidelity: 99.166  -  Fidelity (norm): 261.410
7: Fidelity: 99.166  -  Fidelity (norm): 487.824
8: Fidelity: 99.166  -  Fidelity (norm): 385.580
6: Fidelity: 99.166  -  Fidelity (norm): 289.976
9: Fidelity: 99.166  -  Fidelity (norm): 250.749
10: Fidelity: 99.166  -  Fidelity (norm): 263.756
11: Fidelity: 99.166  -  Fidelity (norm): 445.047
13: Fidelity: 99.166  -  Fidelity (norm): 273.265
12: Fidelity: 99.166  -  Fidelity (norm): 292.892
14: Fidelity: 99.166  -  Fidelity (norm): 329.810
16: Fidelity: 99.166  -  Fidelity (norm): 316.287
15: Fidelity: 99.166  -  Fidelity (norm): 358.612
17: Fidelity: 99.166  -  Fidelity (norm): 349.882
18: Fidelity: 99.166  -  Fidelity (norm): 285.253
19: Fidelity: 99.166  -  Fidelity (norm): 

In [11]:
exps_leg = [exp['leg'] for exp in Exps]
utils.display_results(exps_leg, metrics, agg='mean')
utils.display_results(exps_leg, metrics, agg='median')

,leg,shd,fscore,err,acyc,time
0,PGD-FISTA,253.4000 ± 29.7143,0.5344 ± 0.0535,0.7105 ± 0.1118,0.1254 ± 0.0488,2.6960 ± 0.7227
1,PGD-Adam,253.8500 ± 30.3105,0.5356 ± 0.0506,0.7057 ± 0.1032,0.1057 ± 0.0568,5.3317 ± 1.1677
2,MM,258.7500 ± 30.7032,0.5176 ± 0.0618,0.6925 ± 0.1105,0.0062 ± 0.0070,112.9181 ± 19.5707
3,MM-FISTA,264.4500 ± 31.3249,0.5084 ± 0.0614,0.7555 ± 0.1199,0.0011 ± 0.0013,10.3720 ± 2.5195
4,MM-adam,265.7500 ± 31.4339,0.4981 ± 0.0636,0.7962 ± 0.1371,0.0154 ± 0.0097,29.0397 ± 4.1902
5,MM-adam-fix,342.9500 ± 27.1946,0.2981 ± 0.0496,1.3114 ± 0.1312,0.0239 ± 0.0037,31.9700 ± 4.0376
6,PGD-matexp,275.7500 ± 34.9469,0.4948 ± 0.0720,0.7405 ± 0.1401,0.0000 ± 0.0000,11.5283 ± 0.0910
7,MM-fista-matexp,402.0000 ± 23.8390,0.1386 ± 0.0306,1.6572 ± 0.0659,0.0000 ± 0.0000,31.5132 ± 2.3199
8,CoLiDe,478.0500 ± 21.3107,0.1477 ± 0.0171,1.7226 ± 0.0345,0.0000 ± 0.0000,20.0232 ± 2.0900
9,DAGMA,374.3000 ± 22.3497,0.2597 ± 0.0334,1.4199 ± 0.0772,0.0000 ± 0.0000,19.9054 ± 2.8576


,leg,shd,fscore,err,acyc,time
0,PGD-FISTA,245.0000 ± 29.7143,0.5425 ± 0.0535,0.6830 ± 0.1118,0.1345 ± 0.0488,2.4898 ± 0.7227
1,PGD-Adam,246.5000 ± 30.3105,0.5508 ± 0.0506,0.6899 ± 0.1032,0.0932 ± 0.0568,5.0686 ± 1.1677
2,MM,249.5000 ± 30.7032,0.5352 ± 0.0618,0.6713 ± 0.1105,0.0031 ± 0.0070,106.9917 ± 19.5707
3,MM-FISTA,259.5000 ± 31.3249,0.5169 ± 0.0614,0.7422 ± 0.1199,0.0007 ± 0.0013,8.6840 ± 2.5195
4,MM-adam,264.0000 ± 31.4339,0.4991 ± 0.0636,0.7703 ± 0.1371,0.0135 ± 0.0097,29.3821 ± 4.1902
5,MM-adam-fix,339.0000 ± 27.1946,0.3002 ± 0.0496,1.3286 ± 0.1312,0.0252 ± 0.0037,33.3247 ± 4.0376
6,PGD-matexp,275.0000 ± 34.9469,0.4839 ± 0.0720,0.7551 ± 0.1401,0.0000 ± 0.0000,11.5287 ± 0.0910
7,MM-fista-matexp,395.0000 ± 23.8390,0.1343 ± 0.0306,1.6657 ± 0.0659,0.0000 ± 0.0000,31.1374 ± 2.3199
8,CoLiDe,473.5000 ± 21.3107,0.1455 ± 0.0171,1.7182 ± 0.0345,0.0000 ± 0.0000,19.3596 ± 2.0900
9,DAGMA,366.5000 ± 22.3497,0.2628 ± 0.0334,1.4223 ± 0.0772,0.0000 ± 0.0000,20.4074 ± 2.8576


In [12]:
###  1000 samples - 100 nodes  #####
N = 100

n_dags = 20
data_params = {
    'n_nodes': N,
    'n_samples': 500, # 1000,
    'graph_type': 'er',
    'edges': 2*N,
    'edge_type': 'positive',
    'w_range': (.5, 1),
    'var': 1,
}

print('CPUs employed:', N_CPUS)

t_init = time.time()
results = Parallel(n_jobs=N_CPUS)(delayed(run_exps)(g, data_params, Exps, thr=.2, verb=False) for g in range(n_dags))
ellapsed_time = (time.time() - t_init)/60
print(f'----- Solved in {ellapsed_time:.3f} minutes -----')

# Extract results
shd, fscore, err, acyc, runtime = zip(*results)
metrics = {'shd': shd, 'fscore': fscore, 'err': err, 'acyc': acyc, 'time': runtime}


CPUs employed: 64
0: Fidelity: 99.166  -  Fidelity (norm): 71.628
1: Fidelity: 99.166  -  Fidelity (norm): 75.734
2: Fidelity: 99.166  -  Fidelity (norm): 84.268
3: Fidelity: 99.166  -  Fidelity (norm): 83.625
4: Fidelity: 99.166  -  Fidelity (norm): 80.132
6: Fidelity: 99.166  -  Fidelity (norm): 76.063
5: Fidelity: 99.166  -  Fidelity (norm): 87.529
8: Fidelity: 99.166  -  Fidelity (norm): 75.754
7: Fidelity: 99.166  -  Fidelity (norm): 74.576
11: Fidelity: 99.166  -  Fidelity (norm): 73.160
9: Fidelity: 99.166  -  Fidelity (norm): 80.430
10: Fidelity: 99.166  -  Fidelity (norm): 79.408
12: Fidelity: 99.166  -  Fidelity (norm): 74.125
13: Fidelity: 99.166  -  Fidelity (norm): 80.141
14: Fidelity: 99.166  -  Fidelity (norm): 75.489
15: Fidelity: 99.166  -  Fidelity (norm): 74.159
16: Fidelity: 99.166  -  Fidelity (norm): 84.033
19: Fidelity: 99.166  -  Fidelity (norm): 79.910
17: Fidelity: 99.166  -  Fidelity (norm): 86.702
18: Fidelity: 99.166  -  Fidelity (norm): 82.586
----- Solved

In [13]:
# M = 1000
exps_leg = [exp['leg'] for exp in Exps]
utils.display_results(exps_leg, metrics, agg='mean')
utils.display_results(exps_leg, metrics, agg='median')

,leg,shd,fscore,err,acyc,time
0,PGD-FISTA,47.2500 ± 9.1262,0.8185 ± 0.0234,0.4010 ± 0.0478,0.1412 ± 0.0561,1.6265 ± 0.3063
1,PGD-Adam,42.0000 ± 11.1714,0.8448 ± 0.0331,0.3441 ± 0.0543,0.1365 ± 0.0580,1.2905 ± 0.5730
2,MM,42.2500 ± 9.9039,0.8452 ± 0.0278,0.3530 ± 0.0561,0.0067 ± 0.0046,70.3192 ± 9.4395
3,MM-FISTA,48.6000 ± 9.5153,0.8190 ± 0.0232,0.3984 ± 0.0428,0.0026 ± 0.0014,9.6613 ± 1.7436
4,MM-adam,41.6000 ± 7.9586,0.8460 ± 0.0230,0.3678 ± 0.0477,0.0080 ± 0.0064,15.6180 ± 3.7535
5,MM-adam-fix,71.2500 ± 14.4044,0.7396 ± 0.0447,0.4941 ± 0.0886,0.0158 ± 0.0059,21.5608 ± 2.1434
6,PGD-matexp,58.8000 ± 12.0150,0.8035 ± 0.0302,0.4152 ± 0.0525,0.0000 ± 0.0000,10.8379 ± 0.5430
7,MM-fista-matexp,124.9000 ± 20.7434,0.5145 ± 0.0614,0.9120 ± 0.1151,0.0000 ± 0.0000,34.0265 ± 4.6914
8,CoLiDe,215.2500 ± 25.2564,0.3199 ± 0.0457,1.3991 ± 0.0889,0.0000 ± 0.0000,16.1153 ± 1.2285
9,DAGMA,136.0000 ± 18.1659,0.5198 ± 0.0433,1.0014 ± 0.0660,0.0000 ± 0.0000,14.3777 ± 2.5808


,leg,shd,fscore,err,acyc,time
0,PGD-FISTA,47.0000 ± 9.1262,0.8164 ± 0.0234,0.3889 ± 0.0478,0.1331 ± 0.0561,1.5296 ± 0.3063
1,PGD-Adam,42.0000 ± 11.1714,0.8447 ± 0.0331,0.3460 ± 0.0543,0.1200 ± 0.0580,1.0056 ± 0.5730
2,MM,43.0000 ± 9.9039,0.8388 ± 0.0278,0.3537 ± 0.0561,0.0054 ± 0.0046,68.7338 ± 9.4395
3,MM-FISTA,48.5000 ± 9.5153,0.8134 ± 0.0232,0.3944 ± 0.0428,0.0028 ± 0.0014,8.6330 ± 1.7436
4,MM-adam,41.5000 ± 7.9586,0.8424 ± 0.0230,0.3653 ± 0.0477,0.0043 ± 0.0064,15.9765 ± 3.7535
5,MM-adam-fix,71.0000 ± 14.4044,0.7434 ± 0.0447,0.4862 ± 0.0886,0.0159 ± 0.0059,21.5879 ± 2.1434
6,PGD-matexp,59.0000 ± 12.0150,0.7944 ± 0.0302,0.4103 ± 0.0525,0.0000 ± 0.0000,10.7240 ± 0.5430
7,MM-fista-matexp,127.5000 ± 20.7434,0.5202 ± 0.0614,0.9158 ± 0.1151,0.0000 ± 0.0000,34.8639 ± 4.6914
8,CoLiDe,215.0000 ± 25.2564,0.3337 ± 0.0457,1.3879 ± 0.0889,0.0000 ± 0.0000,15.8243 ± 1.2285
9,DAGMA,136.5000 ± 18.1659,0.5142 ± 0.0433,0.9979 ± 0.0660,0.0000 ± 0.0000,14.5154 ± 2.5808
